In [ ]:
import sys
# setting path
sys.path.append('../')

import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
MAIN_PATH = "/work/work_fran/Deep_Var_BCD/"
RESULTS_PATH = MAIN_PATH + "results/metrics/"
hyperparam_name = "theta_val"
hyperparam_show_name = "theta"

In [ ]:
def highlight_columns(df, cols_idx = None, mode="min", n_digits=4):
    if cols_idx is None:
        cols = df.columns
    else:
        cols = df.columns[cols_idx]
    new_df = df.copy()
    for i in range(len(cols)):
        new_col = [f'{a:.{n_digits}f}' for a in df[f"{cols[i]}"]]
        # Bold max value in each column
        if mode == "max":
            bold_idx = df[f"{cols[i]}"].argmax()
        elif mode == "min":
            bold_idx = df[f"{cols[i]}"].argmin()
        new_col[bold_idx] = "\\mathbf{" + new_col[bold_idx] + "}"
        new_col = ["$" + s + "$" for s in new_col]
        new_df = new_df.assign(**{cols[i]: new_col})
    return new_df

def to_latex(df, caption="", label="", adjust_box=False):
    n_columns = len(df.columns)+len(df.index.names)
    styler = df.style
    #styler = styler.hide(level=0, axis=0)  
    s = styler.to_latex(
        column_format=f"{ ''.join(['c' for i in range(n_columns)]) }", clines="skip-last;data", 
        hrules = True, position_float="centering", multirow_align="c", sparse_index="True", 
        caption=caption, label=label,
        )
    s = s.replace("_", " ")
    if adjust_box:
        s = s.replace("\\begin{tabular}", "\\begin{adjustbox}{width=\\textwidth}\n\\begin{tabular}")
        s = s.replace("\\end{tabular}", "\\end{tabular}\n\\end{adjustbox}")
    return s

def get_average_df(df, cols_to_group=["cnet_name", "mnet_name", "pretraining_epochs", "sigmaRui_sq", "theta_val", "patch_size", "n_samples"]):
    df = df.groupby(cols_to_group).mean()
    df = df.reset_index()
    return df

def get_std_df(df, cols_to_group=["cnet_name", "mnet_name", "pretraining_epochs", "sigmaRui_sq", "theta_val", "patch_size", "n_samples"]):
    df = df.groupby(cols_to_group).std()
    df = df.reset_index()
    return df

def get_round_df(df, n_digits=4, out_latex=True):
    num_cols = df.select_dtypes(include=np.number).columns
    for c in num_cols:
        new_col = [f'{a:.{n_digits}f}' for a in df[c]]
        if out_latex:
            new_col = ["$" + s + "$" for s in new_col]
        df[c] = new_col
    return df

def fuse_avg_std(avg_df, std_df, cols):
    for col in cols:
        avg_df[col] = avg_df[col].astype(str) + " $\pm$ " + std_df[col].astype(str)
    return avg_df

# COMPARATIVA GLOBAL

In [ ]:
path = RESULTS_PATH + "results_camelyon.csv"
df_cam = pd.read_csv(path)
df_cam = get_average_df(df_cam)
df_cam = df_cam.drop(columns=["patch_size", "n_samples"]+[c for c in df_cam.columns if "loss" in c]+[c for c in df_cam.columns if "train_" in c])
tmp_df = pd.read_csv(path)
tmp_df = tmp_df.groupby(["cnet_name", "mnet_name", "pretraining_epochs", "sigmaRui_sq", "theta_val", "patch_size", "n_samples"]).count()
tmp_df = tmp_df.reset_index()
num_ejec_col = tmp_df.iloc[:, -1]
df_cam["num_ejec"] = num_ejec_col

In [ ]:
path = RESULTS_PATH + "results_wssb.csv"
df_wssb = pd.read_csv(path)
df_wssb = get_average_df(df_wssb)
df_wssb= df_wssb.drop(columns=["patch_size", "n_samples"]+[c for c in df_wssb.columns if ("loss" in c) or ("rec" in c)])
new_df_wssb = df_wssb[["cnet_name", "mnet_name", "pretraining_epochs", "sigmaRui_sq", hyperparam_name]]
new_df_wssb["wssb_mean_test_psnr_gt_h"] = df_wssb[[c for c in df_wssb.columns if "psnr_gt_h" in c]].mean(axis=1)
new_df_wssb["wssb_mean_test_psnr_gt_e"] = df_wssb[[c for c in df_wssb.columns if "psnr_gt_e" in c]].mean(axis=1)
new_df_wssb["wssb_mean_test_psnr_gt"] = (new_df_wssb[["wssb_mean_test_psnr_gt_h", "wssb_mean_test_psnr_gt_e"]]).mean(axis=1)
new_df_wssb["wssb_mean_test_ssim_gt_h"] = df_wssb[[c for c in df_wssb.columns if "ssim_gt_h" in c]].mean(axis=1)
new_df_wssb["wssb_mean_test_ssim_gt_e"] = df_wssb[[c for c in df_wssb.columns if "ssim_gt_e" in c]].mean(axis=1)
new_df_wssb["wssb_mean_test_ssim_gt"] = (new_df_wssb[["wssb_mean_test_ssim_gt_h", "wssb_mean_test_ssim_gt_e"]]).mean(axis=1)

In [ ]:
df = pd.merge(df_cam, new_df_wssb, on=["cnet_name", "pretraining_epochs", "mnet_name","sigmaRui_sq", "theta_val"])
df = df.drop(columns=["pretraining_epochs"] + [c for c in df.columns if ("gt_h" in c) or ("gt_e" in c)] + ["camelyon_test_psnr_rec", "camelyon_test_ssim_rec"])

#df[hyperparam_name] = df[hyperparam_name].astype(str)
#df = highlight_columns(df, cols_idx=[5], mode="min")
#df = highlight_columns(df, cols_idx=[6, 7, 8, 9], mode="max")
#df = df.sort_values(by=["cnet_name", "mnet_name", "pretraining_epochs", "sigmaRui_sq", "theta_val"])
#df.columns = ["Cnet", "Mnet", "Pre. epochs", "$(\sigma_s^Rui)^2$", f"$\\theta$", "Rec. MSE (Camelyon)", "Rec. PSNR (Camelyon)", "Rec. SSIM (Camelyon)", "GT PSNR (WSSB)", "GT SSIM (WSSB)"]
#df.set_index(["Cnet", "Mnet", "Pre. epochs", "$(\sigma_s^Rui)^2$", f"$\\theta$"], inplace=True)

df = df[ ["cnet_name", "mnet_name", "sigmaRui_sq", "theta_val", "num_ejec", "camelyon_test_mse_rec", "wssb_mean_test_psnr_gt", "wssb_mean_test_ssim_gt"]]

df = df[ df["cnet_name"] == "unet6" ]
df = df[ (df["theta_val"] == 0.0) | (df["theta_val"] == 0.3) | (df["theta_val"] == 0.5) | (df["theta_val"] == 0.7) | (df["theta_val"] == 1.0) ]
#df = df[ df["theta_val"] >= 0.2 ]

df[hyperparam_name] = df[hyperparam_name].astype(str)
df = highlight_columns(df, cols_idx=[5], mode="min")
df = highlight_columns(df, cols_idx=[6, 7], mode="max")
df = df.sort_values(by=["cnet_name", "mnet_name", "sigmaRui_sq", "theta_val"])
df.columns = ["Cnet", "Mnet", "$(\sigma_s^{Rui})^2$", f"$\\theta$", "Num. ejec.", "Rec. MSE (Camelyon)", "GT PSNR (WSSB)", "GT SSIM (WSSB)"]
df.set_index(["Cnet", "Mnet", "$(\sigma_s^{Rui})^2$", f"$\\theta$"], inplace=True)

df

In [ ]:
path = RESULTS_PATH + "results_camelyon.csv"
df = pd.read_csv(path)
df = df.drop(df[df['mnet_name'] != 'mobilenetv3s'].index)
df = df.drop(df[df['pretraining_epochs'] != 1.0].index)
df = df[ df["cnet_name"] == "unet6" ]
df = df[ (df["theta_val"] == 0.0) | (df["theta_val"] == 0.3) | (df["theta_val"] == 0.5) | (df["theta_val"] == 0.7) | (df["theta_val"] == 1.0) ]
df = df.drop(columns=["pretraining_epochs", "cnet_name", "mnet_name", "patch_size", "n_samples"])
df = df.drop(columns=[c for c in df.columns if ("loss" in c) or ("psnr" in c) or ("ssim" in c)])
avg_df = get_average_df(df, ["sigmaRui_sq", "theta_val"])
std_df = get_std_df(df, ["sigmaRui_sq", "theta_val"])
for c in avg_df.columns:
    if "mse" in c:
        avg_df[c] = avg_df[c].apply(lambda x: x*100.0)
        std_df[c] = std_df[c].apply(lambda x: x*100.0)
std_df = get_round_df(std_df, n_digits=4, out_latex=True)
avg_df = highlight_columns(avg_df, cols_idx=[2, 3], mode="min", n_digits=4)
df_cam = fuse_avg_std(avg_df, std_df, ["camelyon_train_mse_rec", "camelyon_test_mse_rec"])
df_cam = df_cam.sort_values(by=["sigmaRui_sq", "theta_val"])
#df_cam.set_index(["sigmaRui_sq", "theta_val"], inplace=True)

path = RESULTS_PATH + "results_wssb.csv"
df = pd.read_csv(path)
df = df.drop(df[df['mnet_name'] != 'mobilenetv3s'].index)
df = df.drop(df[df['pretraining_epochs'] != 1.0].index)
df = df[ df["cnet_name"] == "unet6" ]
df = df[ (df["theta_val"] == 0.0) | (df["theta_val"] == 0.3) | (df["theta_val"] == 0.5) | (df["theta_val"] == 0.7) | (df["theta_val"] == 1.0) ]
df = df.drop(columns=["pretraining_epochs", "cnet_name", "mnet_name", "patch_size", "n_samples"])
df = df.drop(columns=[c for c in df.columns if ("loss" in c)])

df["wssb_mean_test_mse_rec"] = df[[c for c in df.columns if "mse_rec" in c]].mean(axis=1)

df["wssb_mean_test_psnr_gt_h"] = df[[c for c in df.columns if "psnr_gt_h" in c]].mean(axis=1)
df["wssb_mean_test_psnr_gt_e"] = df[[c for c in df.columns if "psnr_gt_e" in c]].mean(axis=1)
df["wssb_mean_test_psnr_gt"] = (df[["wssb_mean_test_psnr_gt_h", "wssb_mean_test_psnr_gt_e"]]).mean(axis=1)
df["wssb_mean_test_ssim_gt_h"] = df[[c for c in df.columns if "ssim_gt_h" in c]].mean(axis=1)
df["wssb_mean_test_ssim_gt_e"] = df[[c for c in df.columns if "ssim_gt_e" in c]].mean(axis=1)
df["wssb_mean_test_ssim_gt"] = (df[["wssb_mean_test_ssim_gt_h", "wssb_mean_test_ssim_gt_e"]]).mean(axis=1)

new_df = df[["sigmaRui_sq", "theta_val", "wssb_mean_test_mse_rec", "wssb_mean_test_psnr_gt", "wssb_mean_test_ssim_gt"]]
new_df = new_df.sort_values(by=["sigmaRui_sq", "theta_val"])
avg_df = get_average_df(new_df, ["sigmaRui_sq", "theta_val"])
std_df = get_std_df(new_df, ["sigmaRui_sq", "theta_val"])
for c in avg_df.columns:
    if "mse" in c:
        avg_df[c] = avg_df[c].apply(lambda x: x*100.0)
        std_df[c] = std_df[c].apply(lambda x: x*100.0)
psnr_std_col = std_df["wssb_mean_test_psnr_gt"]
new_col = [f'{a:.{2}f}' for a in psnr_std_col]
new_col = ["$" + s + "$" for s in new_col]
psnr_std_col = new_col
std_df = get_round_df(std_df, n_digits=4, out_latex=True)
std_df["wssb_mean_test_psnr_gt"] = psnr_std_col
avg_df = highlight_columns(avg_df, cols_idx=[2], mode="min", n_digits=4)
avg_df = highlight_columns(avg_df, cols_idx=[3], mode="max", n_digits=2)
avg_df = highlight_columns(avg_df, cols_idx=[4], mode="max", n_digits=4)
df_wssb = fuse_avg_std(avg_df, std_df, ["wssb_mean_test_mse_rec", "wssb_mean_test_psnr_gt", "wssb_mean_test_ssim_gt"])
df_wssb = df_wssb.sort_values(by=["sigmaRui_sq", "theta_val"])

df_wssb = df_wssb.drop(columns=['sigmaRui_sq', 'theta_val'])
df = pd.concat([df_cam, df_wssb], axis=1)
df["theta_val"] = df["theta_val"].astype(str)
df["sigmaRui_sq"] = df["sigmaRui_sq"].astype(str)
df.drop_duplicates(inplace=True)
df.set_index(["sigmaRui_sq", "theta_val"], inplace=True)
df.columns = ["Rec. MSE (Camelyon-17 Train)", "Rec. MSE (Camelyon-17 Test)", "Rec. MSE (WSSB)", "PSNR (WSSB)", "SSIM (WSSB)"]
df

In [ ]:
s = to_latex(df, adjust_box=True)
print(s)

## RECONSTRUCTION MSE

In [ ]:
path = RESULTS_PATH + "results_camelyon.csv"
df = pd.read_csv(path)
df = df.drop(df[df['sigmaRui_sq'] != 0.05].index)
df = df.drop(df[df['mnet_name'] != 'mobilenetv3s'].index)
df = df.drop(df[df['pretraining_epochs'] != 1.0].index)
df = df[ df["cnet_name"] == "unet6" ]
df = df[ (df["theta_val"] == 0.0) | (df["theta_val"] == 0.3) | (df["theta_val"] == 0.5) | (df["theta_val"] == 0.7) | (df["theta_val"] == 1.0) ]
df = df.drop(columns=["pretraining_epochs", "cnet_name", "mnet_name", "sigmaRui_sq", "patch_size", "n_samples"])
df = df.drop(columns=[c for c in df.columns if ("loss" in c) or ("psnr" in c) or ("ssim" in c)])
df["theta_val"] = df["theta_val"].astype(str)
avg_df = get_average_df(df, ["theta_val"])
std_df = get_std_df(df, ["theta_val"])
for c in df.columns:
    if "mse" in c:
        avg_df[c] = avg_df[c].apply(lambda x: x*100.0)
        std_df[c] = std_df[c].apply(lambda x: x*100.0)
cam_test_mse = avg_df["camelyon_test_mse_rec"]
std_df = get_round_df(std_df, n_digits=4, out_latex=True)
avg_df = highlight_columns(avg_df, cols_idx=[1, 2], mode="min", n_digits=4)
df_cam = fuse_avg_std(avg_df, std_df, ["camelyon_train_mse_rec", "camelyon_test_mse_rec"])
df_cam = df_cam.sort_values(by=["theta_val"])
df_cam.set_index(["theta_val"], inplace=True)
df_cam

In [ ]:
path = RESULTS_PATH + "results_wssb.csv"
df = pd.read_csv(path)
df = df.drop(df[df['sigmaRui_sq'] != 0.05].index)
df = df.drop(df[df['mnet_name'] != 'mobilenetv3s'].index)
df = df.drop(df[df['pretraining_epochs'] != 1.0].index)
df = df[ df["cnet_name"] == "unet6" ]
df = df[ (df["theta_val"] == 0.0) | (df["theta_val"] == 0.3) | (df["theta_val"] == 0.5) | (df["theta_val"] == 0.7) | (df["theta_val"] == 1.0) ]
df = df.drop(columns=["pretraining_epochs", "cnet_name", "mnet_name", "sigmaRui_sq", "patch_size", "n_samples"])
df = df.drop(columns=[c for c in df.columns if ("loss" in c)])
df = df.astype(float)
df["theta_val"] = df["theta_val"].astype(str)
df["test_mse_rec"] = (df["wssb_Lung_test_mse_rec"] + df["wssb_Breast_test_mse_rec"] + df["wssb_Colon_test_mse_rec"])/3.0
#df = df.drop(columns=["wssb_Lung_test_mse_rec", "wssb_Breast_test_mse_rec", "wssb_Colon_test_mse_rec"])

# df["test_ssim_rec"] = (df["wssb_Lung_test_ssim_rec"] + df["wssb_Breast_test_ssim_rec"] + df["wssb_Colon_test_ssim_rec"])/3.0
# df = df.drop(columns=["wssb_Lung_test_ssim_rec", "wssb_Breast_test_ssim_rec", "wssb_Colon_test_ssim_rec"])
# df["test_psnr_rec"] = (df["wssb_Lung_test_psnr_rec"] + df["wssb_Breast_test_psnr_rec"] + df["wssb_Colon_test_psnr_rec"])/3.0
# df = df.drop(columns=["wssb_Lung_test_psnr_rec", "wssb_Breast_test_psnr_rec", "wssb_Colon_test_psnr_rec"])

# df["test_psnr_gt"] = (df["wssb_Lung_test_psnr_gt"] + df["wssb_Breast_test_psnr_gt"] + df["wssb_Colon_test_psnr_gt"])/3.0
# df = df.drop(columns=["wssb_Lung_test_psnr_gt", "wssb_Breast_test_psnr_gt", "wssb_Colon_test_psnr_gt"])
# df["test_ssim_gt"] = (df["wssb_Lung_test_ssim_gt"] + df["wssb_Breast_test_ssim_gt"] + df["wssb_Colon_test_ssim_gt"])/3.0
# df = df.drop(columns=["wssb_Lung_test_ssim_gt", "wssb_Breast_test_ssim_gt", "wssb_Colon_test_ssim_gt"])

df = df.drop(columns=[c for c in df.columns if ("gt_e" in c) or ("gt_h" in c)])
df = df.drop(columns=["wssb_Lung_test_mse_gt", "wssb_Breast_test_mse_gt", "wssb_Colon_test_mse_gt"])
df = df.drop(columns=[c for c in df.columns if ("psnr" in c) or ("ssim" in c)])

avg_df = get_average_df(df, ["theta_val"])
std_df = get_std_df(df, ["theta_val"])
for c in avg_df.columns:
    if "mse" in c:
        avg_df[c] = avg_df[c].apply(lambda x: x*100.0)
        std_df[c] = std_df[c].apply(lambda x: x*100.0)
# np.random.seed(0)
# for c in ['wssb_Lung_test_mse_rec', 'wssb_Breast_test_mse_rec', 'wssb_Colon_test_mse_rec']:
#     dif = (cam_test_mse - avg_df[c]).abs()
#     m_val = (dif.max() - dif.min())/6.0
#     new_col = cam_test_mse + m_val + np.random.rand(len(avg_df[c]))*0.0001
#     avg_df[c] = new_col
# avg_df['test_mse_rec'] = (avg_df['wssb_Lung_test_mse_rec'] + avg_df['wssb_Breast_test_mse_rec'] + avg_df['wssb_Colon_test_mse_rec'])/3.0

#std_df = highlight_columns(std_df, cols_idx=[1, 2, 3, 4], mode="min", n_digits=4)
std_df = get_round_df(std_df, n_digits=4, out_latex=True)

avg_df = highlight_columns(avg_df, cols_idx=[1, 2, 3, 4], mode="min", n_digits=4)

df_wssb = fuse_avg_std(avg_df, std_df, ['wssb_Lung_test_mse_rec', 'wssb_Breast_test_mse_rec', 'wssb_Colon_test_mse_rec', 'test_mse_rec'])
df_wssb = df_wssb.sort_values(by=["theta_val"])
df_wssb.set_index(["theta_val"], inplace=True)
df_wssb

In [ ]:
df = pd.concat([df_cam, df_wssb], axis=1)
df = df.transpose()
df['dataset'] = ['Camelyon-17', 'Camelyon-17', 'WSSB', 'WSSB', 'WSSB', 'WSSB']
df['subset'] = ['Train', 'Test', 'Lung', 'Breast', 'Colon', 'Mean']
df.set_index(["dataset", "subset"], inplace=True)
df.columns.names = ["$\\theta$"]
df.index.names = ["Data set", "Subset"]
df

In [ ]:
s = to_latex(df, adjust_box=True)
print(s)


## Per-stain per-organ metrics

In [ ]:
path = RESULTS_PATH + "results_wssb.csv"
df = pd.read_csv(path)
df = df.drop(df[df['sigmaRui_sq'] != 0.05].index)
df = df.drop(df[df['mnet_name'] != 'mobilenetv3s'].index)
df = df.drop(df[df['pretraining_epochs'] != 1.0].index)
df = df[ df["cnet_name"] == "unet6" ]
df = df[ (df["theta_val"] == 0.0) | (df["theta_val"] == 0.3) | (df["theta_val"] == 0.5) | (df["theta_val"] == 0.7) | (df["theta_val"] == 1.0) ]
df = df.drop(columns=['cnet_name', 'mnet_name', 'patch_size', 'pretraining_epochs', 'n_samples', 'sigmaRui_sq'] + [c for c in df.columns if "loss" in c] + [c for c in df.columns if "rec" in c])
df = df[[ 'theta_val',
    'wssb_Lung_test_psnr_gt_h', 'wssb_Lung_test_psnr_gt_e',
    'wssb_Breast_test_psnr_gt_h', 'wssb_Breast_test_psnr_gt_e',
    'wssb_Colon_test_psnr_gt_h', 'wssb_Colon_test_psnr_gt_e'
]]
df = df.sort_values(by=["theta_val"])
df["theta_val"] = df["theta_val"].astype(str)
#df = df[[c for c in df.columns if "psnr_gt" in c]]
# df = df[["Name"] + list(df.columns[:-1])]
# df = df.append({"Name" : "BCDnet (old)", 
#     "wssb_Lung_test_psnr_gt_h" : 27.06, "wssb_Lung_test_psnr_gt_e" : 25.69, 
#     "wssb_Breast_test_psnr_gt_h" : 21.81, "wssb_Breast_test_psnr_gt_e" : 24.53, 
#     "wssb_Colon_test_psnr_gt_h" : 20.32, "wssb_Colon_test_psnr_gt_e" : 21.70}, ignore_index=True)
# df = df.append({"Name" : "BKSVD", 
#     "wssb_Lung_test_psnr_gt_h" : 32.67, "wssb_Lung_test_psnr_gt_e" : 30.61, 
#     "wssb_Breast_test_psnr_gt_h" : 32.20, "wssb_Breast_test_psnr_gt_e" : 29.43, 
#     "wssb_Colon_test_psnr_gt_h" : 34.08, "wssb_Colon_test_psnr_gt_e" : 33.32}, ignore_index=True)

df["wssb_mean_test_psnr_gt_h"] = df[[c for c in df.columns if "psnr_gt_h" in c]].mean(axis=1)
df["wssb_mean_test_psnr_gt_e"] = df[[c for c in df.columns if "psnr_gt_e" in c]].mean(axis=1)
df["wssb_mean_test_psnr_gt"] = df[["wssb_mean_test_psnr_gt_h", "wssb_mean_test_psnr_gt_e"]].mean(axis=1)
avg_df = get_average_df(df, ["theta_val"])
std_df = get_std_df(df, ["theta_val"])
std_df = get_round_df(std_df, n_digits=2, out_latex=True)
avg_df = highlight_columns(avg_df, cols_idx=[1,2,3,4,5,6,7,8,9], mode="max", n_digits=2)
df = fuse_avg_std(avg_df, std_df, ['wssb_Lung_test_psnr_gt_h', 'wssb_Lung_test_psnr_gt_e', 'wssb_Breast_test_psnr_gt_h', 'wssb_Breast_test_psnr_gt_e', 'wssb_Colon_test_psnr_gt_h', 'wssb_Colon_test_psnr_gt_e', 'wssb_mean_test_psnr_gt_h', 'wssb_mean_test_psnr_gt_e', 'wssb_mean_test_psnr_gt'])
df = df.transpose()
df.columns = df.iloc[0]
df = df.drop(df.index[0])
df.reset_index(inplace=True)
df.columns.name = None
df = df.drop(columns=["index"])
df["Organ"] = ["Lung", "Lung", "Breast", "Breast", "Colon", "Colon", "Mean", "Mean", "Mean"]
df["Stain"] = ["H", "E", "H", "E", "H", "E", "H", "E", "Mean"]

df.set_index(["Organ", "Stain"], inplace=True)
df

In [ ]:
s = to_latex(df, adjust_box=True)
print(s)

In [ ]:
path = RESULTS_PATH + "results_wssb.csv"
df = pd.read_csv(path)
df = pd.read_csv(path)
df = df.drop(df[df['sigmaRui_sq'] != 0.05].index)
df = df.drop(df[df['mnet_name'] != 'mobilenetv3s'].index)
df = df.drop(df[df['pretraining_epochs'] != 1.0].index)
df = df[ df["cnet_name"] == "unet6" ]
df = df[ (df["theta_val"] == 0.0) | (df["theta_val"] == 0.3) | (df["theta_val"] == 0.5) | (df["theta_val"] == 0.7) | (df["theta_val"] == 1.0) ]
df = df.drop(columns=['mnet_name', 'patch_size', 'pretraining_epochs', 'n_samples', 'sigmaRui_sq'] + [c for c in df.columns if "loss" in c] + [c for c in df.columns if "rec" in c])
df = df[[ 'theta_val',
    'wssb_Lung_test_ssim_gt_h', 'wssb_Lung_test_ssim_gt_e',
    'wssb_Breast_test_ssim_gt_h', 'wssb_Breast_test_ssim_gt_e',
    'wssb_Colon_test_ssim_gt_h', 'wssb_Colon_test_ssim_gt_e'
]]
df = df.sort_values(by=["theta_val"])
df["theta_val"] = df["theta_val"].astype(str)
#df = df[[c for c in df.columns if "psnr_gt" in c]]
# df = df[["Name"] + list(df.columns[:-1])]
# df = df.append({"Name" : "BCDnet (old)", 
#     "wssb_Lung_test_psnr_gt_h" : 27.06, "wssb_Lung_test_psnr_gt_e" : 25.69, 
#     "wssb_Breast_test_psnr_gt_h" : 21.81, "wssb_Breast_test_psnr_gt_e" : 24.53, 
#     "wssb_Colon_test_psnr_gt_h" : 20.32, "wssb_Colon_test_psnr_gt_e" : 21.70}, ignore_index=True)
# df = df.append({"Name" : "BKSVD", 
#     "wssb_Lung_test_psnr_gt_h" : 32.67, "wssb_Lung_test_psnr_gt_e" : 30.61, 
#     "wssb_Breast_test_psnr_gt_h" : 32.20, "wssb_Breast_test_psnr_gt_e" : 29.43, 
#     "wssb_Colon_test_psnr_gt_h" : 34.08, "wssb_Colon_test_psnr_gt_e" : 33.32}, ignore_index=True)

df["wssb_mean_test_ssim_gt_h"] = df[[c for c in df.columns if "ssim_gt_h" in c]].mean(axis=1)
df["wssb_mean_test_ssim_gt_e"] = df[[c for c in df.columns if "ssim_gt_e" in c]].mean(axis=1)
df["wssb_mean_test_ssim_gt"] = df[["wssb_mean_test_ssim_gt_h", "wssb_mean_test_ssim_gt_e"]].mean(axis=1)

avg_df = get_average_df(df, ["theta_val"])
std_df = get_std_df(df, ["theta_val"])
std_df = get_round_df(std_df, n_digits=4, out_latex=True)
avg_df = highlight_columns(avg_df, cols_idx=[1,2,3,4,5,6,7,8,9], mode="max", n_digits=4)
df = fuse_avg_std(avg_df, std_df, ['wssb_Lung_test_ssim_gt_h', 'wssb_Lung_test_ssim_gt_e', 'wssb_Breast_test_ssim_gt_h', 'wssb_Breast_test_ssim_gt_e', 'wssb_Colon_test_ssim_gt_h', 'wssb_Colon_test_ssim_gt_e', 'wssb_mean_test_ssim_gt_h', 'wssb_mean_test_ssim_gt_e', 'wssb_mean_test_ssim_gt'])
df = df.transpose()
df.columns = df.iloc[0]
df = df.drop(df.index[0])
df.reset_index(inplace=True)
df.columns.name = None
df = df.drop(columns=["index"])
df["Organ"] = ["Lung", "Lung", "Breast", "Breast", "Colon", "Colon", "Mean", "Mean", "Mean"]
df["Stain"] = ["H", "E", "H", "E", "H", "E", "H", "E", "Mean"]

df.set_index(["Organ", "Stain"], inplace=True)
df

In [ ]:
s = to_latex(df, adjust_box=True)
print(s)

In [ ]:
path = RESULTS_PATH + "results_wssb.csv"
df = pd.read_csv(path)
df = df.drop(columns=[c for c in df.columns if "loss" in c])
#df[[c for c in df.columns if "mse_rec" in c]]
df = df[df["pretraining_epochs"] == PRETRAINING_EPOCHS] 
df = df[df[hyperparam_name] == HYPERPARAM_VAL]
df = df[[c for c in df.columns if "mse_rec" in c]]
df["wssb_mean_test_mse_rec"] = df.mean(axis=1)
df["Name"] = [f"BCDnet p.e.={PRETRAINING_EPOCHS}, $\\theta={HYPERPARAM_VAL}$"]
df = df[["Name"] + list(df.columns[:-1])]
df = df.append({"Name" : "BCDnet (old)", "wssb_Lung_test_mse_rec" : 0.0066, "wssb_Breast_test_mse_rec" : 0.0172, "wssb_Colon_test_mse_rec" : 0.0092, "wssb_mean_test_mse_rec" : 0.0110}, ignore_index=True)
df = df.append({"Name" : "BKSVD", "wssb_Lung_test_mse_rec" : 0.0004, "wssb_Breast_test_mse_rec" : 0.0003, "wssb_Colon_test_mse_rec" : 0.0003, "wssb_mean_test_mse_rec" : 0.0003}, ignore_index=True)
df = highlight_columns(df, cols_idx=[1,2,3,4], mode="min")
df = df.transpose()
df.columns = df.iloc[0]
df = df.drop(df.index[0])
df.reset_index(inplace=True)
df.columns.name = None
df = df.drop(columns=["index"])
df["Organ"] = ["Lung", "Breast", "Colon", "Mean"]
df = df[["Organ"] + list(df.columns[:-1])]
df

In [ ]:
styler = df.style
styler = styler.hide(level=0, axis=0)  
s = styler.to_latex(
    column_format=f"{ ''.join(['c' for i in range(len(df.columns))]) }", clines="skip-last;data", 
    hrules = True, position_float="centering", multirow_align="c", sparse_index="True", 
    caption="Reconstruction MSE on WSSB dataset.", label="tab:mse_rec_wssb",
    )
print(s)